# Семинар 8 - Сверточные нейронные сети (Convolution neural network)

In [ ]:
import random
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

In [ ]:
from keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print('В трейне: ', x_train.shape[0], 'объектов, в тесте: ', x_test.shape[0])

In [ ]:
num_classes = 10 #
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Сгенерируем случаные примеры для каждого класса
fig = plt.figure(figsize=(8,3))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = x_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = np.transpose(features_idx[img_num,::],(0,1,2))
    ax.set_title(class_names[i])
    plt.imshow(im)
plt.show()

In [ ]:
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=1000, train_size=49000)

In [ ]:
print('Training data shape: ', x_train.shape)
print('Validation data shape: ', x_val.shape)
print('Test data shape: ', x_test.shape)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from keras.models import Sequential # Модель, где все слои соединены друг с другом
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten # Слой, где все нйероны предыдущего уровня соединены с нейронами следующего
from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.optimizers import SGD

In [ ]:
np.random.seed(18)

x_train = x_train.astype('float32')
x_val= x_val.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_val /= 255
x_test /= 255

In [ ]:
y_train_ohe = np_utils.to_categorical(y_train, num_classes=num_classes)
y_val_ohe = np_utils.to_categorical(y_val, num_classes=num_classes)
y_test_ohe = np_utils.to_categorical(y_test, num_classes=num_classes)

__Baseline:__

- Convolutional input layer, 32 feature maps with a size of 3×3, a rectifier activation function and a weight constraint of max norm set to 3.
- Dropout set to 20%.
- Convolutional layer, 32 feature maps with a size of 3×3, a rectifier activation function and a weight constraint of max norm set to 3.
- Max Pool layer with size 2×2.
- Flatten layer.
- Fully connected layer with 512 units and a rectifier activation function.
- Dropout set to 50%.
- Fully connected output layer with 10 units and a softmax activation function.

In [ ]:
epochs = 3
lrate = 0.01
decay = lrate/epochs

In [ ]:
# Создаем последовательную модель
model = Sequential()

# Добавляем слои
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


# Компилируем модель
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

In [ ]:
# Обучаем модель! 
model.fit(x_train, y_train_ohe,
              epochs=epochs,
              validation_data=(x_val, y_val_ohe),
              shuffle=True)

In [ ]:
#Сохраним модель и веса
with open('cnn.json', 'w') as model_file:
            model_file.write(model.to_json())

model.save_weights('cnn.h5')

In [ ]:
from keras.models import model_from_json
json_file = open("cnn.json", "r")
loaded_model_json = json_file.read()
json_file.close()
# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model.load_weights("cnn.h5")

In [ ]:
predict = loaded_model.predict(x_test)

In [ ]:
# Проверим случаный семпл:
num_pred = np.random.randint(0,999)
plt.title(class_names[np.where(y_test_ohe[num_pred]==1)[0][0]])
plt.imshow(x_test[num_pred])
plt.show()
for i,j in enumerate(class_names):
    print(j, predict[num_pred][i])

In [ ]:
accuracy_score(y_test, loaded_model.predict_classes(x_test))

In [ ]:
from keras import Model

In [ ]:
# Функция для отрисовки фильтров
def plot_conv_weights(model, layer_name):
    W = model.get_layer(name=layer_name).get_weights()[0]
    if len(W.shape) == 4:
        W = np.squeeze(W)
        W = W.reshape((W.shape[0], W.shape[1], W.shape[2]*W.shape[3])) 
        fig, axs = plt.subplots(4,8, figsize=(16,8))
        fig.subplots_adjust(hspace = .5, wspace=.001)
        axs = axs.ravel()
        for i in range(32):
            axs[i].imshow(W[:,:,i])
            axs[i].set_title(str(i+1))

In [ ]:
plot_conv_weights(loaded_model, 'conv2d_3')

In [ ]:
activation_model = Model(inputs=loaded_model.input, 
                         outputs=loaded_model.layers[0].output)
activation_model.summary()

In [ ]:
test_img = np.expand_dims(x_test[num_pred], axis=0)
activation = activation_model.predict(test_img)
print(activation.shape)

In [ ]:
plt.subplot(121)
plt.title(class_names[np.where(y_test_ohe[num_pred]==1)[0][0]])
plt.imshow(x_test[num_pred])
plt.subplot(122)
plt.imshow(activation[0, :, :, 6], cmap='viridis')
plt.show()